In [ ]:
# lstm for time series forecasting
from numpy import sqrt
from numpy import asarray
from pandas import read_csv
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [ ]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return asarray(X), asarray(y)

In [ ]:
# load the dataset
path = 'https://raw.githubusercontent.com/snpsuen/Deep_Learning_JupyterLab/main/data/hko_tc8_1970-2022.csv'
df = read_csv(path, header=0, index_col=0, squeeze=True)
# retrieve the values
values = df.values.astype('int32')
# specify the window size
n_steps = 4
# split into samples
X, y = split_sequence(values, n_steps)
print("X = ", X)
print("y = ", y)
print("X.shape = %s, y.shape = %s" %(X.shape, y.shape))
# reshape into [samples, timesteps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))
# split into train/test
n_test = 10
X_train, X_test, X_predict, y_train, y_test, y_predict = X[:-n_test], X[-n_test:-1], X[-1:], y[:-n_test], y[-n_test:-1], y[-1:]
print("After reshape, X_train.shape = %s, X_test.shape = %s, X_predict.shape = %s, y_train.shape = %s, y_test.shape = %s, y_predict.shape = %s" %(X_train.shape, X_test.shape, X_predict.shape, y_train.shape, y_test.shape, y_predict.shape))
print("X_test = ", X_test)
print("X_predict = ", X_predict)
input_shape = X.shape[1:]
print("input_shape = ", input_shape)

X =  [[2 3 1 1]
 [3 1 1 2]
 [1 1 2 1]
 [1 2 1 2]
 [2 1 2 1]
 [1 2 1 2]
 [2 1 2 2]
 [1 2 2 1]
 [2 2 1 1]
 [2 1 1 0]
 [1 1 0 2]
 [1 0 2 1]
 [0 2 1 2]
 [2 1 2 2]
 [1 2 2 0]
 [2 2 0 0]
 [2 0 0 2]
 [0 0 2 0]
 [0 2 0 1]
 [2 0 1 1]
 [0 1 1 4]
 [1 1 4 0]
 [1 4 0 3]
 [4 0 3 1]
 [0 3 1 1]
 [3 1 1 0]
 [1 1 0 5]
 [1 0 5 0]
 [0 5 0 2]
 [5 0 2 1]
 [0 2 1 2]
 [2 1 2 1]
 [1 2 1 0]
 [2 1 0 0]
 [1 0 0 1]
 [0 0 1 4]
 [0 1 4 3]
 [1 4 3 0]
 [4 3 0 1]
 [3 0 1 3]
 [0 1 3 2]
 [1 3 2 1]
 [3 2 1 1]
 [2 1 1 2]
 [1 1 2 5]
 [1 2 5 1]
 [2 5 1 1]
 [5 1 1 2]
 [1 1 2 2]]
y =  [2 1 2 1 2 2 1 1 0 2 1 2 2 0 0 2 0 1 1 4 0 3 1 1 0 5 0 2 1 2 1 0 0 1 4 3 0
 1 3 2 1 1 2 5 1 1 2 2 2]
X.shape = (49, 4), y.shape = (49,)
After reshape, X_train.shape = (39, 4, 1), X_test.shape = (9, 4, 1), X_predict.shape = (1, 4, 1), y_train.shape = (39,), y_test.shape = (9,), y_predict.shape = (1,)
X_test =  [[[3]
  [0]
  [1]
  [3]]

 [[0]
  [1]
  [3]
  [2]]

 [[1]
  [3]
  [2]
  [1]]

 [[3]
  [2]
  [1]
  [1]]

 [[2]
  [1]
  [1]
  [2]]

 [[1]
  [

In [ ]:
# define model
model = Sequential()
# model.add(LSTM(100, activation='relu', kernel_initializer='he_normal', input_shape=input_shape))
model.add(LSTM(64, kernel_initializer='he_normal', input_shape=input_shape))
model.add(Dense(32, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 64)                16896     
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
# fit the model
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=2, validation_data=(X_test, y_test))

Epoch 1/10
2/2 - 3s - loss: 2.4970 - mae: 1.1884 - val_loss: 2.6576 - val_mae: 1.1088 - 3s/epoch - 1s/step
Epoch 2/10
2/2 - 0s - loss: 2.1390 - mae: 1.0936 - val_loss: 2.3141 - val_mae: 0.9419 - 53ms/epoch - 27ms/step
Epoch 3/10
2/2 - 0s - loss: 1.8536 - mae: 0.9934 - val_loss: 2.0311 - val_mae: 0.8603 - 42ms/epoch - 21ms/step
Epoch 4/10
2/2 - 0s - loss: 1.6918 - mae: 0.9558 - val_loss: 1.8088 - val_mae: 0.8245 - 32ms/epoch - 16ms/step
Epoch 5/10
2/2 - 0s - loss: 1.5841 - mae: 0.9587 - val_loss: 1.6543 - val_mae: 0.8076 - 35ms/epoch - 17ms/step
Epoch 6/10
2/2 - 0s - loss: 1.5309 - mae: 0.9718 - val_loss: 1.5546 - val_mae: 0.7896 - 32ms/epoch - 16ms/step
Epoch 7/10
2/2 - 0s - loss: 1.5327 - mae: 0.9878 - val_loss: 1.4989 - val_mae: 0.7760 - 31ms/epoch - 16ms/step
Epoch 8/10
2/2 - 0s - loss: 1.5221 - mae: 0.9885 - val_loss: 1.4755 - val_mae: 0.7679 - 36ms/epoch - 18ms/step
Epoch 9/10
2/2 - 0s - loss: 1.5290 - mae: 0.9930 - val_loss: 1.4744 - val_mae: 0.7655 - 38ms/epoch - 19ms/step
Epoch

In [ ]:
# evaluate the model
mse, mae = model.evaluate(X_test, y_test, verbose=0)
print('MSE: %.3f, RMSE: %.3f, MAE: %.3f' % (mse, sqrt(mse), mae))

MSE: 1.490, RMSE: 1.221, MAE: 0.768


In [ ]:
# make a prediction
## row = asarray([5, 1, 1, 2, 2]).reshape((1, n_steps, 1))
## print("row = %s" %(row))
## yhat = model.predict(row)
## print('Predicted: %.3f' % (yhat))
print("X_predict = %s" %(X_predict))
print("X_predict.shape = ", X_predict.shape)
# print("After reshape, X_train.shape = %s, X_test.shape = %s, X_predict.shape = %s, y_train.shape = %s, y_test.shape = %s, y_predict.shape = %s" %(X_train.shape, X_test.shape, X_predict.shape, y_train.shape, y_test.shape, y_predict.shape))
yhat = model.predict(X_predict)
print('Predicted: %.3f' % (yhat))

X_predict = [[[1]
  [1]
  [2]
  [2]]]
X_predict.shape =  (1, 4, 1)
1/1 [==============================] - 0s 463ms/step
Predicted: 1.545


In [ ]:
# save the model
model.save("RNN_TC8_Annually_Local_Model")
print('Model saved as RNN_TC8_Annually_Local_Model')
! tar czf RNN_TC8_Annually_Local_Model.tar.gz RNN_TC8_Annually_Local_Model

Model saved as RNN_TC8_Annually_Local_Model


In [ ]:
# Connect to remote staging repo at Github
! git clone https://github.com/snpsuen/Deep_Learning_JupyterLab.git; \
cd Deep_Learning_JupyterLab; \
git remote remove origin; \
git remote add origin https://my_token-xxxxxxxxxx@github.com/snpsuen/Deep_Learning_JupyterLab.git

Cloning into 'Deep_Learning_JupyterLab'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 49 (delta 9), reused 6 (delta 1), pack-reused 0
Unpacking objects: 100% (49/49), done.


In [ ]:
# Upload the model to the remote repo
! cp RNN_TC8_Annually_Local_Model.tar.gz Deep_Learning_JupyterLab/model; \
cd Deep_Learning_JupyterLab; \
git add . ; \
git config --global user.email "snpsuen@gmail.com"; git commit -m "Upload RNN TC8 model archive"; \
git push -u origin main; git remote remove origin

[main 4919778] Upload RNN TC8 model archive
 1 file changed, 0 insertions(+), 0 deletions(-)
 rewrite model/RNN_TC8_Annually_Local_Model.tar.gz (96%)
Counting objects: 4, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 282.25 KiB | 21.71 MiB/s, done.
Total 4 (delta 0), reused 0 (delta 0)
To https://github.com/snpsuen/Deep_Learning_JupyterLab.git
   2df5162..4919778  main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.


In [ ]:
# Connect to the local runtime 
# fetch the model zip archive from the Github repo and unpack it under a local folder named model
! cd model; \
wget https://github.com/snpsuen/Deep_Learning_JupyterLab/raw/main/model/RNN_TC8_Annually_Local_Model.tar.gz; \
tar xzf RNN_TC8_Annually_Local_Model.tar.gz

--2022-10-13 06:12:44--  https://github.com/snpsuen/Deep_Learning_JupyterLab/raw/main/model/RNN_TC8_Annually_Local_Model.tar.gz
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/snpsuen/Deep_Learning_JupyterLab/main/model/RNN_TC8_Annually_Local_Model.tar.gz [following]
--2022-10-13 06:12:45--  https://raw.githubusercontent.com/snpsuen/Deep_Learning_JupyterLab/main/model/RNN_TC8_Annually_Local_Model.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 290992 (284K) [application/octet-stream]
Saving to: ‘RNN_TC8_Annually_Local_Model.tar.gz’

RNN_TC8_Annually_Lo 100%[===================>] 284.1